In [0]:
!pip install timeout_decorator -q
!pip install wordcloud -q
!pip install language-tool-python -q
!pip install openai==0.28 -q
!pip install databricks-feature-store -q

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.r

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import openai
import pandas as pd
import re
from matplotlib import pyplot as plt
# from wordcloud import WordCloud
from pyspark.sql.functions import *
from pyspark.sql.types import *
# from timeout_decorator import timeout
import time
from databricks.feature_store import FeatureStoreClient
from datetime import datetime
from pytz import timezone
from pyspark.sql import Window
from pyspark.sql.functions import row_number
from itertools import chain
import ast

## Loading the Data

In [0]:
# %sql
# select *
# from abfrl_catalog.ml_tables.gen_ai_product_description
# where a_flagged_style = '0'
# limit 20

In [0]:
%sql
select *
from abfrl_catalog.pfrl_vrdm.rsh_ean_details
limit 5

ProductID,StyleCode,EAN,SIZE,Gender,Category,Brand,ProductName,LongDescription,ShortDescription,ImageURL,ProductLink,a_linkrewrite,EAN_PRICE,HSN_NUMBER,A_ISPUBLISHED,PRODUCT_ACTIVE,a_mpactive,a_sap_material,a_monthandyearofmanufacturer,a_manufacturerinfo,ARTICLECODE,EAN_ACTIVE,a_sap_color,a_categoryid,LiveDate,Accessories,Band_Material,Band_Type,Brand_nm,Brand_Fit,Brand_Fit_Name,Case_Material,Chronograph_1,Collar_1,Collection_1,Color_1,Color_Product_Naming,Cuffs_1,Dial_Color,Dial_Shape,Display_Type,Fit_1,Front_Opening,Hood_1,Length_1,Material_1,Mechanism_1,Neck_1,Occasion_1,Pattern_1,Power_Source,Product_Height,Product_Length,Product_Type,Product_Width,Season_1,Shoe_Type,Shorts_Type,Size_1,Sleeves_1,Sole_Material,Story_1,Strap_Color,Style_1,Sub_brand,Suit_Front,Swatch_Color,Trend_1,Trouser_Front,Upper_Material,USP_1,Wash_1,product_care,Fabric_Type,PRIMARY_COLOR,SubCategory,Waist_Rise,Dispatch_by_Days,DATA_SRC_CD,REC_STRT_DT,REC_END_DT,ETL_JOB_NM,modified_ts
9370,TF110000034007,205000000000019,XXL,Women,Tees & Tops,Pantaloons,Pantaloons Pink Top,"Effortless yet stylish, this pink top from Bare Denim by Pantaloons is perfect to channel a retro look to your off duty style. Made from rayon, it features an all over polka dot print, three-fourth sleeves, and a round neck with a short button placket. Style it with black denims and sandals for a laid back look.","Effortless yet stylish, this pink top from Bare Denim by Pantaloons is perfect to channel a retro look to your off duty style. Made from rayon, it features an all over polka dot print, three-fourth sleeves, and a round neck with a short button placket. Style it with black denims and sandals for a laid back look.",null,https://pantaloons.com/p/pantaloons-pink-top-9370.html,pantaloons-pink-top,699.00000,61012000,0,1,1,null,null,"Aditya Birla Fashion and Retail Limited,Divyashree Technopolis, Ground floor,KH No: 118/110/1, Building 2,Yemalur Post, Off HAL Airport Road,Bangalore-560037",null,1,Pink,202,2015-10-30T12:14:25Z,null,null,null,Pantaloons,null,null,null,null,null,null,Pink,Pink,null,null,null,null,Regular Fit,null,null,null,Rayon,null,Round Neck,Casual,Print,null,null,null,Top,null,1502,null,null,null,3/4th Sleeves,null,null,null,null,Bare Denim,null,#ffffff,null,null,null,null,null,null,null,Pink,null,null,null,1000,2024-09-27,null,RSH PRODUCT DETAILS,2024-09-27T12:10:49.764Z
31588,TH301171308005,205000004452034,38,Men,Shirts,Pantaloons,Black Textured Evening Full Sleeves Regular Collar Men Slim Fit,"Men's party wear scene just got hotter with F Factor by Pantaloons. Get your bling on with contemporary evening wear, making way for opulence and shimmer.",null,null,https://pantaloons.com/p/black-textured-party-shirt-31588.html,black-textured-party-shirt,1199.00000,61012000,0,1,1,null,null,"Aditya Birla Fashion and Retail Limited,Divyashree Technopolis, Ground floor,KH No: 118/110/1, Building 2,Yemalur Post, Off HAL Airport Road,Bangalore-560037",null,1,Black,187,2015-04-08T20:40:12Z,null,null,null,Pantaloons,null,Slim Fit,null,null,Regular Collar,null,Black,null,null,null,null,null,Slim Fit,null,null,null,Cotton Polyester Blend,null,Regular Collar,Evening,Textured,null,null,null,null,null,1402,null,null,null,Full Sleeves,null,null,null,null,F Factor,null,null,null,null,null,null,null,null,null,Black,null,null,null,1000,2024-09-27,null,RSH PRODUCT DETAILS,2024-09-27T12:10:49.764Z
30007,BY301173951016,205000004454900,32,Men,Trousers & Chinos,Pantaloons,Navy Blue Business Casual Trouser,"Structured, clean and impeccably stylish, Byford by Pantaloons, offers British inspired design elements with a preppy touch that adds to the contemporary man's sporty image. Comprising sophisticated dual toned t-shirts, polos with badges, sweatshirts, and checkered shirts;the quintessential must-haves in every man's wardrobe, the clothes ensure that they never fail to impress.",null,null,https://pantaloons.com/p/navy-blue-business-casual-trouser-30007.html,navy-blue-business-casual-trouser,

In [0]:
# %sql
# select distinct StyleCode, LongDescription, ShortDescription, ProductLink 
# from abfrl_catalog.pfrl_vrdm.rsh_ean_details
# where brand = 'Pantaloons'
# and Gender = 'Men'
# and modified_ts >= (current_timestamp() - interval 24 hour)
# and LiveDate >= (current_timestamp() - interval 1 month)
# and sub_brand = 'Byford'
# and Category = 'T-Shirts'
# and PRODUCT_ACTIVE = '1'
# and StyleCode in ('BY110110312210')

In [0]:
df = spark.sql('''
select distinct StyleCode as Style_code, 
                Gender,
                REPLACE(Product_Type, '&amp;', 'and') as Category,
                Sub_brand as Brand,
                ProductName as Short_Description,
                regexp_replace(ShortDescription, '\\d+|%|&#\d+;|<p>|</p>|<b>|</b>|cm|&nbsp|<p class="description">|;', '') AS Description,
                REPLACE(Accessories,  '&amp;', 'and') as Accessory_Type, -- Replacing `&`` with `&`
                REPLACE(Collection_1, '&amp', 'and') as Collection,
                Color_1 as Color, 
                Cuffs_1 as Cuff,
                Fit_1 as Fit,
                Front_Opening,
                REGEXP_REPLACE(
                  REPLACE(
                    REPLACE(
                      REPLACE(
                        REGEXP_REPLACE(coalesce(Upper_Material, Material_1), '[0-9%.-]', ''),
                        '&amp;', ','),
                     ' and ', ' , '),
                   '&comma;', ''), 
                '\\belas[^ ]*\\b', 'elastane') AS Material,
                Neck_1 as Neck,
                Occasion_1 as Occasion,
                Pattern_1 as Pattern,
                Power_Source,
                Shoe_Type,
                Shorts_Type,
                Sleeves_1 as Sleeve,
                Sole_Material,
                Strap_Color,
                Style_1 as Style,
                regexp_replace(Suit_Front, 'Single Breasted - ', '') as Suit_front,
                Trouser_Front,
                Wash_1 as Wash,
                Waist_Rise
from abfrl_catalog.pfrl_vrdm.rsh_ean_details
where Brand='Pantaloons'
and StyleCode in ('TA11012707357', 'TA110127074227', 'BY11012149720', 'TJ110124687313')
               ''')

In [0]:
# df.display()
# df = df.limit(15)

In [0]:
# to check fo a particular brand/product/ etc.
# df.filter(df.Material.contains('Polyester,  Viscose And  Elastic')).display()

In [0]:
# df.select('Neck').distinct().display()

In [0]:
source_count = df.select("Style_Code").distinct().count()
source_count

4

In [0]:
source = df.select("Style_Code").distinct()

In [0]:
df = df.select('Style_Code',"Gender", "Color", "Material", "Fit", "Occasion","Neck","Pattern",'Sleeve','Cuff',"Front_Opening",'Sole_Material','Wash','Category','Brand','Description','Power_Source','Shorts_Type','Shoe_Type','Accessory_Type','Waist_Rise','Suit_front','Trouser_Front','Strap_Color','Short_Description','collection')
df = df.fillna('')

In [0]:
# df.display()

## Preprocessing & Cleaning the Data

In [0]:
# brand level cleaning i.e. changing the names to a standardised name
df = df.withColumn('brand',when(col('brand')=='LP','Louis Philippe')
                        .when(col('brand')=='VH','Van Heusen')
                        .when(col('brand')=='AS','Allen Solly')
                        .when(col('brand')=='LP Jeans','Louis Philippe Jeans')
                        .when(col('brand')=='Tribe','Allen Solly Tribe')
                        .when(col('brand')=='V Dot', 'V Dot by Van Heusen')
                        .when(col('brand')=='Vdot', 'V Dot by Van Heusen')   #addition
                        .when(col('brand')=='Allen Solly Jeans','Allen Solly')
                        .when(col('brand')=='Allen Solly Woman','Allen Solly')
                        .when(col('brand')=='Van Heusen Woman','Van Heusen')
                        .when(col('brand')=='Van Heusen Woman','Van Heusen')
                        .otherwise(col('brand')))

In [0]:
# category level cleaning i.e. changing the names based on Front_Opening
# changes to be made based on PT data
df = df.withColumn('Front_Opening', 
                   when(((initcap(col('Category'))=='Jeans')
                        |(initcap(col('Category'))=='Trousers')
                        |(initcap(col('Category'))=='Skirt')
                        |(initcap(col('Category'))=='Shorts')
                        |(initcap(col('Category'))=='Pants')
                        #|(initcap(col('Category'))=='Antiviral Jeans')
                        |(initcap(col('Category'))=='Bermuda')
                        |(initcap(col('Category'))=='Cargo')) 
                        &((initcap(col('Front_Opening'))=='Zip Front')
                        |(initcap(col('Front_Opening'))=='Zip-Button Front')), None)
                        .otherwise(col('Front_Opening')))

In [0]:
#Pattern level cleaning i.e. changing names based on Pattern
# changes to be made based on PT data
df = df.withColumn('Pattern', when(initcap(col('Pattern')).isin('Ouch','Flat Front','Crop Tops','Patterned'), None)\
                              .when(lower(col('Pattern'))=='floral,floral,floral', 'floral')\
                              .when(lower(col('Pattern'))=='print-tropical', 'tropical')\
                              .when(initcap(col('Pattern'))=='Colorblock', 'color block')\
                              .when(initcap(col('Pattern'))=='Striped', 'stripe')\
                              .when(initcap(col('Pattern'))=='Checks', 'Check')\
                              .when(initcap(col('Pattern'))=='Checked', 'Check')\
                              .when(initcap(col('Pattern'))=='Solids', 'solid')\
                              .when(initcap(col('Pattern'))=='Animal Printed', 'animal print')\
                              .when(initcap(col('Pattern'))=='Dots', 'dot')\
                   .otherwise(col('Pattern')))

In [0]:
#not needed for PT
# df = df.withColumn('Color', when(initcap(col('Color'))=='Trasparent', 'transparent').otherwise(col('Color')))

In [0]:
# add Material's substitute values
df = df.withColumn("Material", trim(regexp_replace(col("Material"), "^/", "")))\
.withColumn("Material", regexp_replace(col("Material"), "/", ","))\
.withColumn('Material', when(upper(col('Material'))=='PU', 'vegan leather')\
  .when(lower(col('Material'))=='polyurethane and  polyester', 'vegan leather and polyester')\
  .when(col('Material')=='Cotton,  Naylon and  Spandex', 'Cotton,  Nylon and  Spandex')\
  .when(lower(col('Material'))=='Lather', 'Leather')\
  .when(upper(col('Material'))=='CVC', 'cotton and polyester')\
  .when(initcap(col('Material'))=='Polyurethane Leather', 'vegan leather')\
  .when(initcap(col('Material'))=='Terylene Rayon', 'Terylene Rayon')\
  .when(initcap(col('Material'))=='Polyurethane And  Tc', 'vegan leather and polyester')\
  .when(initcap(col('Material'))=='Cotton,  Naylon And  Spandex', 'Cotton,  Nylon and  Spandex')\
  .when(initcap(col('Material'))=='Micro Fibro', 'Micro Fiber')\
  .when(initcap(col('Material'))=='Tie &amp; Pocket Square   Polyester', 'Polyester')\
  .when(initcap(col('Material'))=='Polyester,  Viscose And  Elastic', 'Polyester,  Viscose and  Elastane')\
  .when(initcap(col('Material'))=='Polyester Viscose Elasti', 'Polyester Viscose Elastane')\
  .when(initcap(col('Material'))==')Polyester, Rayon And  Elastine', 'Polyester, Rayon and  Elastane')\
  .when(lower(col('Material'))=='polyester and  viscose and   elastene', 'Polyester and  Viscose and   Elastane')\
  .when(lower(col('Material'))=='polyester, viscose and  elastien', 'Polyester, Viscose and  Elastane')\
  .when(lower(col('Material'))=='p,olyester,  viscose and  spandex', ' Polyester,  Viscose and  Spandex')\
  .when(lower(col('Material'))=='cotton and  polyester and  stretch', 'elastane - multiple ends')\
  .when(initcap(col('Material'))=='Poly Cotton Blend', 'polyester and cotton')\
  .when(lower(col('Material'))=='cotton and  spadnex', 'Cotton and  spandex')\
  .when(initcap(col('Material'))=='Polyamide And Spandex', 'nylon and elastane')\
  .when(lower(col('Material'))=='polyamide and   elastane', 'nylon and elastane')\
  .when(lower(col('Material'))=='cotton,  polyester and  sparkle', 'spandex (for sparkle)')\
  .when(initcap(col('Material'))=='Polyurethane Leather', 'vegan leather')\
  .when(lower(col('Material'))=='polyester, viscose and e lasti', 'elastane')\
  .when(lower(col('Material'))=='terylenerayon', 'terylene and rayon')\
  .when(initcap(col('Material'))=='Micro Fibre', 'Micro Fiber')\
  .when(lower(col('Material'))=='lambswool and  nylon', 'lamb wool')\
  .when(lower(col('Material'))=='pu and cotton blend', 'vegan leather')\
  .when(lower(col('Material'))=='tc,  pu and others', 'vegan leather')\
  .when(initcap(col('Material'))=='Cottom', 'Cotton')\
  .when(lower(col('Material'))=='pu leather', 'vegan leather')\
  .when(lower(col('Material'))=='polyamide and  Eeastane', 'nylon')\
  .when(lower(col('Material'))=='polyamide and  spandex', 'nylon')\
  .when(initcap(col('Material'))=='Polyster  Viscose', 'polyester')\
  .when(initcap(col('Material'))=='Polyurethane', 'vegan leather')\
  .when(lower(col('Material'))==' merino and  acrylic&comma;', 'merino and acrylic')\
  .when(initcap(col('Material'))=='Polyurethane', 'vegan leather')\
  .when(initcap(col('Material'))=='Synthetic', None)\
  .when(initcap(col('Material'))=='Synthetic Leather', 'vegan leather')\
  .when(lower(col('Material'))=='polyurethane and mesh', 'vegan leather')\
  .when(lower(col('Material'))=='polyurethane and textile', 'vegan leather')\
  .when(initcap(col('Material'))=='Weaved Tape', None)\
  .when(initcap(col('Material'))=='Weaved Tape', None)\
  .when(lower(col('Material'))=='fabric', None)\
  .when(lower(col('Material'))=='cloth cast llp sitapur industrial area, g, phaseiii jaipur ', None)\
  .when(lower(col('Material'))=='contemporary exports llp , kamal v complex , mathura road sector  ,  faridabad , hariyana', None)\
  .when(initcap(col('Material'))=='Accessory', None)\
  .when(initcap(col('Material'))=='Polyuethane', 'vegan leather')\
  .when(lower(col('Material'))=='knit and polyurethane', 'Knit and Vegan Leather')\
  .when(lower(col('Material'))=='flyknit and polyurethane', 'Flyknit and Vegan Leather')\
#-----Adding Spelling Rectifications----#
  .when(initcap(col('Material'))==' Cotton ,  Elastanee', 'Cotton ,  Elastane')\
  .when(initcap(col('Material'))==' Cotton Polyester Elastan', 'Cotton, Polyester, Elastane')\
  .when(initcap(col('Material'))==' Nylon ,  Elastanee', 'Nylon ,  Elastane')\
  .when(lower(col('Material'))==' cotton,  nylon an  spandex', 'cotton, nylon, spandex')\
  .when(initcap(col('Material'))==" Cotton,  Polyester ,  ElastaneSuper Slim Fit Jeans", 'cotton,  polyester ,  elastane')\
  .when(initcap(col('Material'))==' Cotton,  Nylon ,  Elastanee', 'cotton, nylon, elastane')\
  .when(initcap(col('Material'))==' Polyester Viscose Spande', ' polyester, viscose, spandex')\
  .when(initcap(col('Material'))=='` Cotton', 'cotton')\
  .when(initcap(col('Material'))=='Polyester  Spandex', 'polyester, spandex')\
  .when(initcap(col('Material'))=='Poly Nylon Blend', 'polyester, nylon blend')\
  .when(initcap(col('Material'))==' Polyester,  Viscose  Cotton ,  Spandex', ' polyester,  viscose, cotton ,  spandex')\
  .when(initcap(col('Material'))==' Cotton+ Spandex', ' cotton, spandex')\
  .when(initcap(col('Material'))==' Cotton,  Elastane ,  ElastaneSuper Slim Fit Jeans', 'cotton, elastane')\
  .when(initcap(col('Material'))==' Cotton ,  Ployester', 'cotton, polyester')\
  .when(initcap(col('Material'))==' Cotton ,  Ployester', 'cotton, polyester')\
  .when(initcap(col('Material'))=='^ Cotton', 'cotton')\
  .when(upper(col('Material')).like('%PU%'), 'vegan leather')\
  .when(initcap(col('Material'))=='c  p   el', 'cotton polyester spandex')\
  .when(initcap(col('Material'))=='cotton  poly  lycra  to  ozs option', 'cotton polyester spandex')\
  .when(initcap(col('Material'))=='co  poly  spdx', 'cotton polyester spandex')\
  .when(initcap(col('Material'))=='c  p  vis ray  el', 'cotton polyester viscose rayon spandex')\
  .when(initcap(col('Material'))=='cotton  spandex sj', 'cotton spandex')\
  .when(initcap(col('Material'))=='c  p  vis,ray  el', 'cotton polyester viscose rayon spandex')\
  .when(initcap(col('Material'))=='cotton  spenx', 'cotton  spandex')\
  .when(initcap(col('Material'))=='ctn tencil spandex', 'cotton tencil spandex')\
  .when(initcap(col('Material'))=='cotton  polyester  elastane   oz', 'cotton  polyester  elastane')\
  .when(lower(col('Material')).isin('other', 'fibers', 'gsm', 'accessory', 'ozs', 'oz', 'tshirt', 'tshirts', 'top', 'tops', 'shirt', 'shirts', 'jacket', 'jackets','aeo stretch boxer short multipack'),'')\
  .otherwise(col('Material')))

In [0]:
df = df.withColumn('Cuff', when(lower(col('Cuff'))=='regular cuff', '').when(lower(col('Cuff'))=='double cuff', 'double').otherwise(col('Cuff')))

In [0]:
df = df.withColumn('Sleeve', regexp_replace(lower(col('Sleeve'))," sleeve",""))\
       .withColumn('Sleeve', regexp_replace(lower(col('Sleeve'))," sleeves",""))

In [0]:
df = df.withColumn('Sleeve', when(lower(col('Sleeve'))=='fulls', 'full')\
  .when(lower(col('Sleeve'))=='longs', 'long')\
  .when(lower(col('Sleeve'))=='halfs', 'half')\
  .when(lower(col('Sleeve'))=='shorts', 'short')\
  .when(lower(col('Sleeve'))=='3/4ths', '3/4th')\
  .when(lower(col('Sleeve'))=='caps', 'cap')\
  .otherwise(lower(col('Sleeve')))
)

In [0]:
df = df.withColumn('Fit', when(initcap(col('Fit'))=='Beat Fit', None).otherwise(col('Fit')))
df = df.withColumn('Fit', when(lower(col('Fit'))=='slim fit', 'slim')\
  .when(lower(col('Fit'))=='regular fit', 'regular')\
  .when(lower(col('Fit'))=='custom fit', 'custom')\
  .when(lower(col('Fit'))=='ultra slim fit', 'ultra slim')\
  .when(lower(col('Fit'))=='snug fit', 'snug')\
  .when(lower(col('Fit'))=='skinny fit', 'skinny')\
  .when(lower(col('Fit'))=='smart fit', 'smart')\
  .when(lower(col('Fit'))=='super slim fit', 'super slim')\
  .when(lower(col('Fit'))=='carrot fit', 'carrot')\
  .when(lower(col('Fit'))=='ankle skinny fit', 'ankle skinny')\
  .when(lower(col('Fit'))=='straight fit', 'straight')\
  .when(lower(col('Fit'))=='sport fit', 'sport')\
  .when(lower(col('Fit'))=='jegging fit', 'jegging')\
  .when(lower(col('Fit'))=='comfort fit', 'comfort')\
  .when(lower(col('Fit'))=='classic fit', 'classic')\
  .when(lower(col('Fit')).isin('original','shirt','fit','t-shirt','v neck','shoulder strap','babydoll','skinny kick','wrap','jeans','tie & dye','tomgirl','jeggings','jegging','shorts','silhouette','three fourth sleeve','sleeveless','short sleeve','kick','textured','solid'),'')\
  .otherwise(lower(col('Fit'))))

In [0]:
df = df.withColumn('Neck', when(initcap(col('Neck'))=='Crew Neck', 'crew')\
  .when(initcap(col('Neck'))=='Polo Neck', 'polo')\
  .when(initcap(col('Neck'))=='Round Neck', 'round')\
  .when(initcap(col('Neck'))=='Stylized Neck', 'stylized')\
  .when(initcap(col('Neck'))=='V Neck', 'v')\
  .when(initcap(col('Neck'))=='Square Neck', 'square')\
  .when(initcap(col('Neck'))=='Hooded Neck', 'hooded')\
  .when(initcap(col('Neck'))=='V-Neck', 'V')\
  .when(initcap(col('Neck'))=='Henley Neck', 'henley')\
  .when(initcap(col('Neck'))=='Slit Neck', 'slit')\
  .when(initcap(col('Neck'))=='Turtle Neck', 'turtle')\
  .when(initcap(col('Neck'))=='Boat Neck', 'boat')\
  .when(initcap(col('Neck'))=='Cowl Neck', 'cowl')\
  .when(initcap(col('Neck'))=='High Neck', 'high')\
  .when(initcap(col('Neck'))=='Sweetheart Neck', 'sweetheart')\
  .when(initcap(col('Neck'))=='Collar Neck', 'collar')\
  .when(initcap(col('Neck'))=='Peterpan Neck', 'peterpan')\
  .when(initcap(col('Neck'))=='Scoop Neck', 'scoop')\
  .when(initcap(col('Neck'))=='Peter Pan Neck', 'peter pan')\
  .when(initcap(col('Neck'))=='Round Crew Neck', 'round crew')\
  .when(initcap(col('Neck'))=='Halter Neck', 'halter')\
  .when(initcap(col('Neck'))=='Mock Neck', 'mock')\
  .otherwise(lower(col('Neck'))
  ))

In [0]:
df = df.withColumn('Sole_Material', when(initcap(col('Sole_Material'))=='Polyurethane', None)\
                   .otherwise(col('Sole_Material')))

In [0]:
df = df.withColumn('Category', when(lower(col('Category'))=='shirts', 'shirt')\
  .when(lower(col('Category'))=='trousers', 'trouser')\
  .when(lower(col('Category'))=='track pants', 'track pant')\
  .when(lower(col('Category'))=='t-shirts', 't-shirt')\
  .when(lower(col('Category'))=='button up shirts', 'button up shirt')\
  .otherwise(lower(col('Category')))
)

In [0]:
columns_to_lower = ['Gender','Category','Accessory_Type', 'Color', 'Cuff', 'Fit', 'Material', 'Neck', 'Occasion', 'Pattern', 'Power_Source', 'Shoe_Type', 'Shorts_Type', 'Sleeve','Sole_Material', 'Strap_Color', 
                   # 'Style',
                     'Suit_Front', 'Trouser_Front', 'Wash', 'Waist_Rise']

for column in columns_to_lower:
    df = df.withColumn(column, lower(col(column)))

In [0]:
df = df.drop_duplicates()

In [0]:
mat_df = df.select('Material')
material_result = mat_df.toPandas()['Material'].unique().tolist()

In [0]:
# mat_df.display()

In [0]:
len(material_result)

2

In [0]:
chunks = int(np.ceil(len(material_result)/100))
print(chunks)

1


## Open-AI Configurations

In [0]:
openai.api_type = "azure"
openai.api_base = "https://abfopenai-analytics.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = '81d383be2dfe43f5aae90e6071ed9fd8'

In [0]:
def get_processed_material(material_list,chunks,chunk_size):
  final_list = []
  for chunk in range(chunks):
      start = chunk*chunk_size
      end = start+chunk_size
      # print(start,end)
      prompt = f"""
                        Instructions:
                        '''
                        - Clean the list of product fabric material values for cataloguing of fashion products on a website.
                        
                        - Ensure the output contains detailed values with no spelling mistakes or short forms.

                        - Ensure the output is in lowercase only apart from acronyms. Eg. 'Cotton' should be 'cotton', 'EVA' should be 'EVA'.

                        - Remove all special characters except comma.

                        - Convert all occurrences of PU, P.U., Polyurethane to 'vegan leather'.

                        - Set all values that are not product materials or abbreviations for product materials to null.   

                        - Refine the list to extract product fabric material names, accurately correcting misspellings. Ensure materials listed together in a single string are separated. For example, 'rayon polyester' becomes 'rayon, polyester', while 'polyester lycra' becomes 'polyester, lycra' or 'line,  cotton' becomes 'cotton'. These examples serve as references; ensure dynamic updates for all strings.

                        - Generate processed product fabric material names for strings containing materials, while assigning null values to strings containing other values. For instance, if the input string is 'rayon polyester', the processed material name will be 'rayon, polyester' while for 'cotton  viscose' output will be 'cotton, viscose'. Similarly, if the string contains a product category like 'tie, pocket square, polyester', the output will focus solely on 'polyester' or if string is 'iron  glass  plastic  polyster' then output will be 'polyester'.


                        - Refine the list to extract only the product fabric names, and strictly omit the occurance of other words like cufflinks, metals, alloys, product categories like shirt, and any other irrelevant values from input string.For example, if the input is "tshirt cotton", the output should be "cotton". Similarly, strings like "polyester, cufflinksteel" should result in "polyester". Ensure that only product fabric names are retained in the output, while all other values are discarded. Additionally, strings like "tie,pocket square polyester,lapel pin, metal, alloy , lapel pinsteel,cufflinks,alloy,metal" should yield "polyester". Ensure dynamic updates for all strings.

                        - Present the output in a list of tuple where the key represents the original input and the value signifies the processed output. Ensure that in the output value, materials separated by commas are unique.

                        - Refine the output to consist solely of a list of tuples without any unnecessary characters like ellipses (...) within the string or in the tuples themselves. Ensure that each tuple contains proper strings without any additional information. For instance, avoid strings like "('ae ne(x)t level..." or [('plastic, glass', None), ...]. It's crucial that the output dynamically adapts to the input data. This refinement is vital to ensure the integrity and usability of the output as a valid list of tuples.

                        - Ensure that each tupple is separated by single comma.
                        
                        '''                     

                        Please process the below given list of product materials according to the instructions above.
                        ```{material_list[start:end]}```                             
                        """

      response = openai.ChatCompletion.create(engine="abfrlgpt",
                                              temperature = 0.2, 
                                              messages=[
                {"role": "system", "content": "Data Preprocessing Specialist for a fashion brand, tasked with correcting the material name by following the instructions enclosed in triple single quotes(''')."},
                {"role": "user", "content": prompt}])
      output = response['choices'][0]['message']['content']


      start = output.find('[')
      end = output.find(']')
      pattern = r"\([^)]*\.\.\.[^)]*\)"
      tuples_with_ellipses = re.findall(pattern, str(output[start:end+1])) 
      filtered_string = re.sub(pattern, '', str(output[start:end+1]))
      output_string = re.sub(r',\s*,',',', filtered_string)
      output_string = output_string.replace("""\n""","")
      print(output_string)
      material_output = ast.literal_eval(output_string)
      final_list = final_list+material_output 

  return final_list

In [0]:
processed_material = get_processed_material(material_result, chunks, 100)
schema = StructType([
    StructField("Material", StringType(), nullable=True),
    StructField("Processed_Material", StringType(), nullable=True)
])
material_output_df = spark.createDataFrame(data=processed_material,schema = schema)

[('cotton', None), ('blended', None)]


In [0]:
# material_output_df.display()

In [0]:
# # missed_materials = list(set(material_output_df.filter(col('Processed_Material').isNull()) \
# #                             .select('Material').rdd.map(lambda row: row.Material).collect()))

# missed_materials = material_output_df.filter(col('Processed_Material').isNull()).select('Material').toPandas()['Material'].unique().tolist()
# chunks = int(np.ceil(len(missed_materials)/20))

# columns = StructType([StructField('Material',StringType(), True),
#                       StructField('Missed_Processed_Material',StringType(), True)])
# missed_materials_output_df = spark.createDataFrame([],schema =  columns)

# retries = 0
# while len(missed_materials)>0 and retries<5:
#   processed_material = get_processed_material(missed_materials,chunks,20)
#   missed_material_df = spark.createDataFrame(data=processed_material,schema= columns)
#   # print('Length of missed material list',len(missed_materials))
#   missed_materials_output_df = missed_materials_output_df.union(missed_material_df)
#   # missed_materials = list(set(missed_material_df.filter(col('Missed_Processed_Material').isNull()) \
#   #                           .select('Material').rdd.map(lambda row: row.Material).collect()))
#   missed_materials = missed_material_df.filter(col('Missed_Processed_Material').isNull()).select('Material').toPandas()['Material'].unique().tolist()
#   retries+=1

In [0]:
import numpy as np
missed_materials = material_output_df.filter(col('Processed_Material').isNull()).select('Material').toPandas()['Material'].unique().tolist()
chunks = int(np.ceil(len(missed_materials)/20))
columns = StructType([StructField('Material',StringType(), True),
                      StructField('Missed_Processed_Material',StringType(), True)])
missed_materials_output_df = spark.createDataFrame([], schema=columns)

retries = 0
while len(missed_materials) > 0 and retries < 5:
    print(f"Retry {retries+1}: Processing {len(missed_materials)} materials")
    processed_material = get_processed_material(missed_materials, chunks, 20)
    
    if processed_material:
        missed_material_df = spark.createDataFrame(data=processed_material, schema=columns)
        missed_materials_output_df = missed_materials_output_df.union(missed_material_df)
        missed_materials = missed_material_df.filter(col('Missed_Processed_Material').isNull()).select('Material').toPandas()['Material'].unique().tolist()
    else:
        print("No data returned from get_processed_material")
        break
    
    retries += 1

print("Final list of missed materials:", missed_materials)

Retry 1: Processing 2 materials
[('cotton', None), ('blended', None)]
Retry 2: Processing 2 materials
[('cotton', None), ('blended', None)]
Retry 3: Processing 2 materials
[('cotton', None), ('blended', None)]
Retry 4: Processing 2 materials
[('cotton', None), ('blended', None)]
Retry 5: Processing 2 materials
[('cotton', None), ('blended', None)]
Final list of missed materials: ['cotton', 'blended']


In [0]:
final_material_df = material_output_df.join(missed_materials_output_df,on='Material',how='left')
final_material_df = final_material_df.withColumn('Processed_Material',coalesce('Processed_Material','Missed_Processed_Material'))
final_material_df = final_material_df.distinct()
material_df = final_material_df.select("Material","Processed_Material")

In [0]:
material_df = material_df.filter("Processed_Material is not null").distinct()

In [0]:
material_df = df.join(material_df,on='Material',how='left')

In [0]:
df = material_df.drop("Material").withColumnRenamed("Processed_Material","Material")

In [0]:
df = df.distinct()

In [0]:
df = df.toPandas()

In [0]:
result = df.apply(lambda row: {key: value for key, value in row.items() if value is not None}, axis=1).tolist()

In [0]:
len(result)

4

## Open-AI Description Generation

In [0]:
def get_description(result):
  final_result = pd.DataFrame()

  missed_stylecodes = []
  for iteration in range(len(result)):
          features = {key:value for key,value in result[iteration].items() if key not in ('Style_Code', 'Short_Description', 'Description', 'product_description')}
          prompt = f"""
                  Instructions:
                  '''
                  - Write a mini-call to action that's both informative and persuasive. What specific benefit will the reader gain by purchasing this product? without using direct call-to-action phrases such as "purchase" or "Shop now".
                  - In up to 65 words using the provided specifications.
                  - Emphasize key features and benefits while adhering to grammatical rules.
                  - Optimize for SEO tools without using emojis or symbols.
                  - Keep the tone inline with the occasion.
                  - Subtly add what to wear it with.
                  - Avoid word repetition.
                  - Make sure the product description is well-formatted and grammatically correct.
                  '''
                  
                  Specifications:
                  ```{features}```
                  """
                  #Write a mini-call to action that's both informative and persuasive.
                  # - Write a comprehensive product description that is comprehensive and informative.
          try:
                response = openai.ChatCompletion.create(engine="abfrlgpt",
                                                        temperature = 1.2,  # previously : 1.2 --> 0.7
                                                        # top_p = 0.1,
                                                        messages=[
                    # Creative content writer/creative copy writer for a premium fasion retail company.
                    {"role": "system", "content": "Marketing Assistant for a fashion brand, tasked with creating website product descriptions by following the instructions enclosed in triple single quotes(''')."},
                    {"role": "user", "content": prompt}])
                output = response['choices'][0]['message']['content']
                result[iteration]['product_description'] = output

                final_result = final_result.append(result[iteration],ignore_index=True)
          except Exception as e:
              missed_stylecodes.append(result[iteration]['Style_Code'])
              print(e)
  
  return (final_result,missed_stylecodes)

In [0]:
descriptions_call = get_description(result)
final_result = descriptions_call[0]
missed_stylecodes = descriptions_call[1]
print('length of stylecodes -',len(final_result),'length of missed styles -',len(missed_stylecodes))
cnt=1
while len(missed_stylecodes)!=0:
    missed_stylecodes_df = df[df['Style_Code'].isin(missed_stylecodes)]
    # missed_stylecodes_dict = missed_stylecodes_df.rdd.map(lambda row: {key: value for key, value in row.asDict().items() if value is not None}).collect()
    missed_stylecodes_dict = missed_stylecodes_df.apply(lambda row: row.dropna().to_dict(), axis=1).tolist()
    descriptions_call = get_description(missed_stylecodes_dict)
    missed_stylecodes_result = descriptions_call[0]
    missed_stylecodes = descriptions_call[1]
    final_result = pd.concat([final_result,missed_stylecodes_result],axis=0)
    print("Iteration -",cnt)
    print('length of stylecodes -',len(missed_stylecodes_result),'length of missed styles -',len(missed_stylecodes))
    print("=======================================")
    cnt+=1

/home/spark-24fc3e68-79a5-4584-aad3-37/.ipykernel/4453/command-512526259464453-2524022986:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_result = final_result.append(result[iteration],ignore_index=True)
/home/spark-24fc3e68-79a5-4584-aad3-37/.ipykernel/4453/command-512526259464453-2524022986:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_result = final_result.append(result[iteration],ignore_index=True)
/home/spark-24fc3e68-79a5-4584-aad3-37/.ipykernel/4453/command-512526259464453-2524022986:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_result = final_result.append(result[iteration],ignore_index=True)


length of stylecodes - 4 length of missed styles - 0


/home/spark-24fc3e68-79a5-4584-aad3-37/.ipykernel/4453/command-512526259464453-2524022986:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_result = final_result.append(result[iteration],ignore_index=True)


In [0]:
# post processing the results
def remove_newline(text_description):
  processed_text = text_description.replace('\n', ' ')
  return processed_text
def multiple_spaces(text_description):
  processed_text = re.sub(' +', ' ', text_description)
  return processed_text
def multiple_dash(text_description):
  processed_text = re.sub('-+', '-', text_description)
  processed_text = re.sub(' -+', '-', text_description)
  processed_text = re.sub('-+ ', '-', text_description)
  processed_text = re.sub(' -+ ', '-', text_description)
  return processed_text

def capitalize_after_full_stop_description(text_description):
  result = ""
  capitalize = False
  for char in text_description:
    if char == ".":
      capitalize = True
      result += char
    elif capitalize and char.isalpha():
      result += char.upper()
      capitalize = False
    else:
      result += char
  return result

# Cleanup driver function
def cleanup_description(description):
  if type(description)!=str:
    return description

    
  brand_list = ['Van Heusen', 'Allen Solly Junior', 'Allen Solly',
       'Van Heusen Sport', 'Allen Solly Sport', 'V Dot by Van Heusen',
       'Van Heusen Denim Labs', 'Van Heusen Flex', 'Allen Solly Tribe',
       'Academy by Van Heusen']
  
  brand_spelling_correction = {'Allan':' Allen',
                               'AllenSolly': ' Allen Solly',
                               ' Soli': ' Solly',
                               ' Soly': ' Solly',
                               ' Alle': ' Allen',
                               ' Alan': ' Allen',
                               ' Ailen': ' Allen',
                               ' Allin': ' Allen',
                               ' Allenn': ' Allen',
                               
                               ' VanHeusen': ' Van Heusen',
                               ' Vann':' Van',
                               ' Venn': ' Van',
                               ' Ven': ' Van',
                               ' Huesen': ' Heusen',
                               ' Housen': ' Heusen',
                               ' Hussen': ' Heusen',
                               ' Hewsen': ' Heusen',
                               ' Huizen': ' Heusen',
                               
                               'LouisPhillippe': 'Louis Phillippe',
                               ' Lewis': ' Louis',
                               ' Loui': ' Louis',
                               ' Loois': ' Louis',
                               ' Lui': ' Louis',
                               ' Luwis': ' Louis',
                               ' Lious': ' Louis',
                               ' LowisPhillipe': 'Louis Phillippe',
                               'Louiss' : "Louis"}
  
  # taking care of Pantaloons with no space
  # description = description.replace('Pantaloons', 'Pantaloons ')
  description = re.sub(r'\bPantaloons(?=\w)', 'Pantaloons ', description)

  # taking care of '!!'
  description = description.replace('!!', '!')

  # checking for spelling mistakes
  for k,v in brand_spelling_correction.items():
    description = description.replace(k, v)

  if 'v dot' in description:
    description = description.replace('v dot', 'V Dot')

  # for brand in brand_list:
  #   small_brand = brand.lower()
  #   if small_brand in description.lower():
  #     for brand_segment in brand.split():
  #       if brand_segment != 'v':
  #         description = description.replace(brand_segment.lower(), brand_segment)

  #   if str('the '+brand) in description:
  #     description = description.replace(str(brand+"'s"), brand)
  #     description = description.replace(str(brand+"’s"), brand)
  #   elif brand in description:
  #     description = description.replace(str(brand+"'s"), 'the '+ brand)
  #     description = description.replace(str(brand+"’s"), 'the '+ brand)
  for brand in brand_list:
    pattern = re.compile(r'\b' + re.escape(brand.lower()) + r'\b', re.IGNORECASE)
    description = pattern.sub(brand, description)

    if 'the ' + brand in description:
        description = description.replace(brand + "'s", brand)
        description = description.replace(brand + "’s", brand)
    elif brand in description:
        description = description.replace(brand + "'s", 'the ' + brand)
        description = description.replace(brand + "’s", 'the ' + brand)
    
  description = description.replace('"""', '')
  description = description.replace('""', '')
  description = description.replace('"', '')

  description = description.replace("'''", '')
  description = description.replace("''", '')
  description = description.replace(" '", '')
  description = description.replace("' ", ' ')
  description = description.replace("’ ", ' ')
  description = description.replace("/", '-')
  # description = description.replace("'", '')

  description = description.replace('```', '')
  description = description.replace('``', '')
  description = description.replace('`', '')

  # description = description.capitalize()

  description = remove_newline(description)
  description = multiple_spaces(description)
  description = multiple_dash(description)
  description = capitalize_after_full_stop_description(description)
  
  return description

In [0]:
def flagging_system(df):
  characters_list = ['+', '_', '(', ')', '{', '}', '[', ']', '~', '`', '=', '$', '#','<', '>','^', '\*', '\|','À', 'à', 'Â', 'â',  'Æ', 'æ', 'Ç', 'ç',
    'É','é', 'È','è', 'Ê','ê', 'Ë','ë',
    'Î','î', 'Ï','ï',
    'Ô','ô', 'Œ','œ',
    'Ù','ù', 'Û','û', 'Ü','ü',
    'Ÿ','ÿ', 'Ã','©', ':', '`', '..', '--', ',,', '!','\\']
  
  num_list = ['1','2','3','4','5','6','7','8','9','0', '/']

  words_list = ['1744', '2-button', '3-button', '2 button', '3 button', '2 Button', '3 Button', '1-button', 
                '1 button', '5 button', '5-button', '1-buttoned', '2-buttoned', '3-buttoned', '1 buttoned', '2 buttoned', '3 buttoned' ,'100%', '4 button', '4-Button', '4-button', '100 %', '3/4', '7x7', '4-buttoned', '4 Buttoned', 'pack of 4', 'pack of 2','forever21','Forever21','5 pack','5-pack',
                '3 pack','3-pack','2 pack','2-pack']

  flag_words_list = ['model', 'Model', 'Buy', 'buy', 'purchase', 'Purchase', 'SEO', 'seo ', 'seo.', 'Shop', 'shop', 'Store', 'store', 'Outlet', 'outlet', 'specification', 'description', 'instruction','Note','Notes','note','notes']

  # skipped_categories = ['belt and wallet', 'wallet and key chain']

  # brand_list = ['Van Heusen', 'Allen Solly']

  # df.dropna(subset = ['product_description'], inplace = True)

  result_dict = {key:[] for key in characters_list+num_list}
  result_dict['short_desc']          = []
  result_dict['long_desc']           = []
  result_dict['flag_words']          = []
  result_dict['flag_brand']          = []
  result_dict['flag_collection']     = []
  result_dict['flag_color']          = []
  result_dict['flag_category']       = []
  result_dict['flag_sleeve']         = []
  result_dict['flag_trouser_front']  = []
  result_dict['flag_fit']            = []
  result_dict['flag_pattern']        = []
  # result_dict['flag_material']       = []
  result_dict['flag_neck']           = []

  counter = 0
  columns = ['Style_Code', 'product_description', 'brand', 'Collection', 'Color', 'Category', 'Fit', 'Sleeve', 'Trouser_Front', 'Pattern', 'Material', 'Neck', 'Accessory_Type']
  for col in columns:
        if col not in df.columns:
            df[col] = None
  
  for data in df[['Style_Code', 'product_description', 'brand', 'Collection', 'Color', 'Category', 'Fit', 'Sleeve', 'Trouser_Front', 'Pattern', 'Material', 'Neck', 'Accessory_Type']].values: # add brand and category
    style_code = data[0]
    description = data[1]
    brand = data[2]
    collection = data[3]
    color = data[4]
    category = data[5].replace('rompers', 'romper')
    fit = data[6]
    sleeve = data[7]
    trouser_front = data[8]
    pattern = data[9]
    # material = data[10]
    neck = data[11]
    # accessories for belt and wallet
    accessory = data[12]

    # tackling the null descriptions
    if type(description)!=str:
      val = result_dict['short_desc']
      val.extend([style_code])
      result_dict['short_desc'] = val
      continue

    # keeping only the exact brand names
    if brand not in description.replace('the ', ''):
      val = result_dict['flag_brand']
      val.extend([style_code])
      result_dict['flag_brand'] = val

    # keeping only the exact collection names
    if type(collection)==str:
      if collection.lower() not in description.lower():
        description = description.replace(collection.lower(), collection)
      elif collection.lower() not in description.lower():
        val = result_dict['flag_collection']
        val.extend([style_code])
        result_dict['flag_collection'] = val

    # keeping only the exact color names
    if type(color)==str:
      if color.lower() not in description.lower():
        val = result_dict['flag_color']
        val.extend([style_code])
        result_dict['flag_color'] = val

  
    if brand not in ('Reebok','American Eagle','Forever21'):
      if type(category)==str:
        if 'wallet and key chain' not in category and 'belt and wallet' not in category:
          category = category.replace('and ', '').split(' ')
          cat_flag = 0
          cufflink_flag = 1
          accessory_flag = 0
          for category_val in category:
            if category_val.strip().replace('-', ' ') not in description.lower().replace('-', ' '):
              cat_flag = 1
              if 'cufflink' in category_val.strip() and 'cufflink' in description.lower():
                cufflink_flag = 0
          if cat_flag and cufflink_flag and 'cufflink' not in category:
            val = result_dict['flag_category']
            val.extend([style_code])
            result_dict['flag_category'] = val



    # flagging if the description is shorter than 50 in length
    if len(description.split(" ")) < 50:
      val = result_dict['short_desc']
      val.extend([style_code])
      result_dict['short_desc'] = val

    # flagging if the description is longer than 90 in length
    if len(description.split(" ")) > 110:
      val = result_dict['long_desc']
      val.extend([style_code])
      result_dict['long_desc'] = val

    # flagging if the description contains the unwanted words
    for flag_word in flag_words_list:
      if flag_word in description:
        val = result_dict['flag_words']
        val.extend([style_code])
        result_dict['flag_words'] = val

    # flagging if the description contains the un-wanted characters
    for character in characters_list:
      if character in description:
        val = result_dict[character]
        val.extend([style_code])
        result_dict[character] = val

    # flagging if the description contains the unwanted numbers but excluding the collection and 'button' types
    for num in num_list:
      if num in description:
        description_temp = description
        for word in words_list:
          if word in description:
            description_temp = description_temp.replace(word, '')
            
        if num in description_temp:
          val = result_dict[num]
          val.extend([style_code])
          result_dict[num] = val

  return result_dict

In [0]:
# flagging_system(temp)
def get_flagged_data(temp):
  sum_ = 0
  unique_ids = []
  for k, v in flagging_system(temp).items():    
    unique_ids.extend(v)
    sum_+=len(v)
  unique_ids = list(set(unique_ids))
  return (sum_,unique_ids)

In [0]:
final_result['product_description'] = final_result['product_description'].apply(lambda x : cleanup_description(x))
# final_result['product_description'] = final_result['product_description'].apply(lambda x : capitalize_after_full_stop_description(x))
temp = final_result.copy(deep=True)
sum_,unique_ids = get_flagged_data(temp)

In [0]:
final_output = temp[~temp['Style_Code'].isin(unique_ids)]

In [0]:
unique_ids_df = df[df['Style_Code'].isin(unique_ids)]

In [0]:
if not unique_ids_df.empty:
  retries = 1
  while retries<=5:
    retry_df = df[df['Style_Code'].isin(unique_ids)]
    # retry_dict = retry_df.rdd.map(lambda row: {key: value for key, value in row.asDict().items() if value is not None}).collect()
    retry_dict = retry_df.apply(lambda row: row.dropna().to_dict(), axis=1).tolist()
    descriptions_call = get_description(retry_dict)
    final_result = descriptions_call[0]
    missed_stylecodes = descriptions_call[1]
    print('length of stylecodes -',len(final_result),'length of missed styles -',len(missed_stylecodes))
    cnt=1
    while len(missed_stylecodes)!=0:
      missed_stylecodes_df = df[df['Style_Code'].isin(missed_stylecodes)]
      # missed_stylecodes_dict = missed_stylecodes_df.rdd.map(lambda row: {key: value for key, value in row.asDict().items() if value is not None}).collect()
      missed_stylecodes_dict = missed_stylecodes_df.apply(lambda row: row.dropna().to_dict(), axis=1).tolist()
      descriptions_call = get_description(missed_stylecodes_dict)
      missed_stylecodes_result = descriptions_call[0]
      missed_stylecodes = descriptions_call[1]
      print("Iteration -",cnt)
      print('length of stylecodes -',len(missed_stylecodes_result),'length of missed styles -',len(missed_stylecodes))
      print("=======================================")
      cnt+=1
      final_result = pd.concat([final_result,missed_stylecodes_result],axis=0)

    final_result['product_description'] = final_result['product_description'].apply(lambda x : cleanup_description(x))
    temp = final_result.copy(deep=True)  
    if retries>4:
      final_output = pd.concat([final_output,temp],axis=0)
    else:
      sum_,unique_ids = get_flagged_data(temp)
      output = temp[~temp['Style_Code'].isin(unique_ids)]
      print("Retries  -",retries)
      print('Length of flagged stylecodes -',len(unique_ids))
      print('===========================')
      final_output = pd.concat([final_output,output],axis=0)  
      if len(unique_ids) ==0:
        break
    retries+=1
else:
  print('no style left')
  pass

no style left


In [0]:
def set_flagged_style(Style):
    if Style in unique_ids:
        return '1'
    else:
        return '0'

# Add flagged_style column based on condition
final_output['a_flagged_style'] = final_output['Style_Code'].apply(lambda x: set_flagged_style(x))

In [0]:
f = spark.createDataFrame(final_output)
f.display()

Style_Code,Gender,Color,Fit,Occasion,Neck,Pattern,Sleeve,Cuff,Front_Opening,Sole_Material,Wash,Category,brand,Description,Power_Source,Shorts_Type,Shoe_Type,Accessory_Type,Waist_Rise,Suit_Front,Trouser_Front,Strap_Color,Short_Description,collection,product_description,Collection,Material,a_flagged_style
BY11012149720,men,black,slim,athleisure,,solid,,,,,,casual trousers,Byford,Be the best you while rocking these trousers with a polo an smart shoes.,,,,,low rise,,,,Black Solid Ankle Length Athleisure Men Slim Fit Casual Trouser,,"Crafted by Byford, these black casual trousers offer a combination of style and comfort. Designed for men, the slim-fit trousers feature a solid pattern with a low-rise waist, making them perfect for athleisure occasions. Pair these trousers with your favourite t-shirt or sweater for a sleek and casual look. Get ready to turn heads and elevate your wardrobe with these eye-catching trousers.",null,null,0
TA11012707357,men,brown,regular,athleisure,round,solid,half,,,,,t-shirt,Ajile,Enjoy your sport to the fullest wearing this T-shirt with joggers an sports shoes.,,,,,,,,,Brown Solid Athleisure Half Sleeves Round Neck Men Regular Fit T-Shirt,,"For men who lead active lifestyles, the Ajile brown t-shirt strikes the right chord. The regular fit provides comfort without compromising on style. With its trendy solid pattern, it effortlessly blends with athleisure outfits or casually paired with jeans. Perfect for those with an eye for fashion and indispensable comfort.",null,null,0
TJ110124687313,women,red,regular,festive,,solid,,,,,,pants,Rangmanch,"These pants with a kurta, a mini bag an fab heels are perfect for a housewarming.",,,,,,,,,Red Solid Blended Women Regular Fit Pant,,"Looking for a ravishing and festive complement to your wardrobe? Rangmanch's women's pants in red with regular fit are perfect for you. A timeless solid pattern ensures easy pairing options. While its premium quality fabric adds effortless elegance, the perfect choice for someone with a discerning taste. Pair it with a crisp lightweight shirt and propel your festivities this season.",null,null,0
TA110127074227,men,green,regular,athleisure,round,solid,half,,,,,t-shirt,Ajile,Gear up for an intense set by wearing this T-shirt with track pants an sports shoes.,,,,,,,,,Green Solid Athleisure Half Sleeves Round Neck Men Regular Fit T-Shirt,,"Stay cool and comfortable while looking stylish in the Ajile men's green t-shirt. The regular fit t-shirt is perfect for the athleisure occasion and is made with a solid pattern, a round neck, and half sleeves. Pair with your favorite jeans for a casual, effortless look. Upgrade your wardrobe today with this versatile green t-shirt from Ajile.",null,null,0


In [0]:
# f = f.select(col("Style_Code").alias("a_stylecode"),
#                                    col("Short_Description").alias("a_shortdescription"),
#                                    col("product_description").alias("a_Longdescription"),col('a_flagged_style'))